In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import pandas as pd
from sklearn.model_selection import train_test_split
import wandb
import matplotlib.pyplot as plt

2024-01-03 02:43:27.647891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/nurul/venvs/tfgpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'probability_NN.ipynb'

In [2]:
# Function to configure GPU
def configure_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
    # Invalid device or cannot modify virtual devices once initialized.
        pass

# Function to load and preprocess data
def load_and_preprocess_data(filepath, test_size=0.2, random_state=42):
    try:
        df = pd.read_csv(filepath)
    except Exception as e:
        print(f"Error reading data file: {e}")
        return None, None, None, None

    y = df['output_val']
    X = df.drop(['output_val'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False, random_state=random_state)
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    return X_train, X_test, y_train, y_test

In [12]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [13]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [16]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [18]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [22]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [19]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

In [23]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2023-12-03 23:55:54.675841: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-12-03 23:55:54.678047: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


17/17 [==============================] - 3s 45ms/step - loss: 26.8218 - root_mean_squared_error: 5.1790 - val_loss: 24.7856 - val_root_mean_squared_error: 4.9785
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 25.0115 - root_mean_squared_error: 5.0012 - val_loss: 23.5564 - val_root_mean_squared_error: 4.8535
Epoch 3/100
17/17 [==============================] - 0s 8ms/step - loss: 23.4477 - root_mean_squared_error: 4.8423 - val_loss: 22.2988 - val_root_mean_squared_error: 4.7222
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 21.9271 - root_mean_squared_error: 4.6826 - val_loss: 20.9946 - val_root_mean_squared_error: 4.5820
Epoch 5/100
17/17 [==============================] - 0s 8ms/step - loss: 20.4346 - root_mean_squared_error: 4.5205 - val_loss: 19.6511 - val_root_mean_squared_error: 4.4330
Epoch 6/100
17/17 [==============================] - 0s 13ms/step - loss: 18.9734 - root_mean_squared_error: 4.3559 - val_loss: 18.2767 - val_root

In [24]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [25]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 147ms/step - loss: 29.5405 - root_mean_squared_error: 5.4339 - val_loss: 28.3167 - val_root_mean_squared_error: 5.3198
Epoch 2/500
5/5 [==============================] - 0s 30ms/step - loss: 28.8743 - root_mean_squared_error: 5.3725 - val_loss: 18.9593 - val_root_mean_squared_error: 4.3532
Epoch 3/500
5/5 [==============================] - 0s 17ms/step - loss: 28.5062 - root_mean_squared_error: 5.3378 - val_loss: 29.5615 - val_root_mean_squared_error: 5.4363
Epoch 4/500
5/5 [==============================] - 0s 17ms/step - loss: 24.2744 - root_mean_squared_error: 4.9253 - val_loss: 24.2946 - val_root_mean_squared_error: 4.9269
Epoch 5/500
5/5 [==============================] - 0s 19ms/step - loss: 26.7932 - root_mean_squared_error: 5.1746 - val_loss: 21.9701 - val_root_mean_squared_error: 4.6860
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 22.0577 - root_mean_squared_error: 4.69

In [ ]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [26]:
def load_and_preprocess_data(filepath, test_size=0.2, random_state=42):
    try:
        df = pd.read_csv(filepath)
    except Exception as e:
        print(f"Error reading data file: {e}")
        return None, None, None, None

    y = df['output_val']
    X = df.drop(['output_val'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False, random_state=random_state)
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    return X_train, X_test, y_train, y_test

data_file_path = './data/train_data_histogram_longtime_210000_coursegrained.csv' # FILE PATH
X_train, X_test, y_train, y_test = load_and_preprocess_data(data_file_path)

X_train shape: (1679200, 2)
y_train shape: (1679200,)


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import layers as tfpl

# Step 1: Dataset Preparation
np.random.seed(0)
t = np.linspace(0, 30, 300)
y = np.sin(t) + np.random.normal(0, 0.2, 300)

train_size = int(len(y) * 0.8)
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Model Architecture
def create_bnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(1,)),
        tfpl.DenseVariational(10, 
                              make_posterior_fn=tfpl.util.default_mean_field_normal_fn(is_singular=False),
                              make_prior_fn=tfpl.util.default_multivariate_normal_fn, 
                              activation='relu'),
        tfpl.DenseVariational(tfpl.IndependentNormal.params_size(1),
                              make_posterior_fn=tfpl.util.default_mean_field_normal_fn(is_singular=False),
                              make_prior_fn=tfpl.util.default_multivariate_normal_fn),
        tfpl.IndependentNormal(1)
    ])
    return model

bnn_model = create_bnn_model()

# Step 3: Training
negloglik = lambda y, rv_y: -rv_y.log_prob(y)
bnn_model.compile(optimizer=tf.optimizers.Adam(), loss=negloglik)

bnn_model.fit(t[:train_size, np.newaxis], y_train, epochs=1000, verbose=False)

# Step 4: Prediction and Uncertainty Quantification
def predict_with_uncertainty(model, t_test, n_iter=100):
    preds = [model(t_test) for _ in range(n_iter)]
    preds = np.concatenate(preds, axis=-1)
    return np.mean(preds, axis=-1), np.std(preds, axis=-1)

mean_pred, std_pred = predict_with_uncertainty(bnn_model, t[train_size:, np.newaxis])

# Plotting the results
import matplotlib.pyplot as plt

plt.fill_between(t[train_size:], mean_pred - 2*std_pred, mean_pred + 2*std_pred, alpha=0.5)
plt.plot(t[train_size:], y_test, label='True')
plt.plot(t[train_size:], mean_pred, label='Predicted')
plt.legend()
plt.show()


TypeError: default_mean_field_normal_fn.<locals>._fn() missing 2 required positional arguments: 'trainable' and 'add_variable_fn'

In [3]:
import torch
import torch.nn as nn
import torchbnn as bnn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
# Function to load and preprocess data
def load_and_preprocess_data(filepath, test_size=0.2, random_state=42):
    try:
        df = pd.read_csv(filepath)
    except Exception as e:
        print(f"Error reading data file: {e}")
        return None, None, None, None

    y = df['R']
    X = df.drop(['output_val', 'R'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False, random_state=random_state)
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    return X_train, X_test, y_train, y_test

In [19]:
data_file_path = './data/train_data_histogram_longtime_210000_coursegrained.csv'
X_train, X_test, y_train, y_test = load_and_preprocess_data(data_file_path)

X_train shape: (1679200, 1)
y_train shape: (1679200,)


In [ ]:
#convert to numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.fit_transform(y_train)
X_test = scaler.fit_transform(X_test)
y_test = scaler.fit_transform(y_test)


# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [23]:

# Define the Bayesian Neural Network
class BNN(nn.Module):
    def __init__(self):
        super(BNN, self).__init__()
        self.fc1 = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1, out_features=1)
        self.fc2 = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1, out_features=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = BNN()

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'bnn_model.pth')


Epoch [10/10000], Loss: 2.0405
Epoch [20/10000], Loss: 1.6224
Epoch [30/10000], Loss: 1.6650
Epoch [40/10000], Loss: 1.7319
Epoch [50/10000], Loss: 2.1938
Epoch [60/10000], Loss: 1.5584
Epoch [70/10000], Loss: 1.6099
Epoch [80/10000], Loss: 1.7315
Epoch [90/10000], Loss: 1.5562
Epoch [100/10000], Loss: 1.8681
Epoch [110/10000], Loss: 1.5563
Epoch [120/10000], Loss: 1.4563
Epoch [130/10000], Loss: 1.4636
Epoch [140/10000], Loss: 1.5692
Epoch [150/10000], Loss: 1.4714
Epoch [160/10000], Loss: 1.4702
Epoch [170/10000], Loss: 1.4404
Epoch [180/10000], Loss: 1.7287
Epoch [190/10000], Loss: 1.3621
Epoch [200/10000], Loss: 1.5469
Epoch [210/10000], Loss: 1.4463
Epoch [220/10000], Loss: 1.2498
Epoch [230/10000], Loss: 1.4419
Epoch [240/10000], Loss: 1.2764
Epoch [250/10000], Loss: 1.3377
Epoch [260/10000], Loss: 1.2815
Epoch [270/10000], Loss: 1.4163
Epoch [280/10000], Loss: 1.3896
Epoch [290/10000], Loss: 1.3620
Epoch [300/10000], Loss: 1.3071
Epoch [310/10000], Loss: 1.4114
Epoch [320/10000]